#Version Checking

In [ ]:
import sys 
import numpy as np
print("Python Version:", sys.version, '\n')
print("Numpy Version: ", np.__version__)

# Why do we like numpy?

Numpy is a great library with many, many functions that extend it well beyond normal Python. We're going to focus on some of the primary functions that we use all the time. 

Also note that Pandas is pretty much just numpy with column names (and some SQL seasoning) in a lot of ways, so anything we can do to a numpy array we can usually do with a Pandas column. A prominent exception: matrix math doesn't work well in pandas.

### Get some data (we'll just use some random data)

In [ ]:
import numpy as np

In [ ]:
data = np.random.uniform(size=10000)
data_as_list = data.tolist()

In [ ]:
data.shape

### Broadcasting

If we want to perform a mathematical operation on a list, we have to explicitly loop through the list. If we want to act on an array, we can just treat it like a single object and the operation is "broadcast" to all of the members of the array. It's LIGHTNING fast. Let's see it in action.

In [ ]:
new_data = data + 2

In [ ]:
for added, raw in zip(new_data[:10],data):
    print(added, raw)

What do I mean when I say "lightning fast"?

In [ ]:
%%timeit
new_list = [x+2 for x in data_as_list]

In [ ]:
%%timeit
new_data2 = data + 2

### Filtering

One of the great things about Pandas is the ability to quickly filter your data. Turns out that's because we're working with numpy arrays. Let's check out filtering on arrays - note that we're going to make a condition and each value in the array will be `True` or `False` based on whether it actually passes the condition.

In [ ]:
new_data > 2.5

Once we have the filter, if we ask the array to adjust itself based on that filter, it will automatically only keep the rows that are True.

In [ ]:
np.sum(new_data > 2.5)

In [ ]:
new_data[new_data > 2.5].shape

In [ ]:
new_data[new_data > 2.5]

We can also filter on any data source that has the same length as the data we're filtering. So for instance, if we make a condition on `data` which is equivalent to `new_data - 2`, we can still filter `new_data` the same way. That's possible because data has the same length.

In [ ]:
new_data[data > 0.5]

#### Exercise:

Write a function that takes in two arrays and a value. The first array is a data array, the second is a 1D array, and the value should be numeric.

We want to use broadcasting and filtering to return the rows of the first array, that correspond to indexes where the second array has a value larger than the user provided value. We also want to add 2 to the values in that row.

Example:

```
x1 = np.array([[1,2],[3,4],[5,6]])
x2 = np.array([1, 2, 3])
value = 1.5

my_function(x1, x2, value)
----
> array([[5, 6],
>        [7, 8]])
```

In [ ]:
# complete exercise here

### Reorganization

`numpy` also allows us to quick reorganize our array. Let's see a simple working example. By providing a list of "how to align the indexes" we can rearrange the data. 

In [ ]:
a = np.array([10,20,30,40,50])
a[[1,0,2,3,4]]

### Random number draws

`numpy` also has a great series of tools for generating random numbers effectively. Let's see a few in action (we'll use matplotlib to visualize.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("seaborn")

def plot_hist(x, bins=None):
    plt.figure(dpi=100)
    plt.hist(x, bins=bins);

In [ ]:
x = np.random.uniform(0,10,size=500)
plot_hist(x)    

In [ ]:
x = np.random.normal(3,1,size=20000)
plot_hist(x, bins=100)

In [ ]:
x = np.random.poisson(2, size=20000)
plot_hist(x, bins=20)

### Matrix Multiplication

Doing matrix multiplication programmatically is usually a hassel. `numpy` makes that simple. There are two methods for doing that: `numpy` treats `@` as meaning "do the matrix multiplication".

In [ ]:
mat_a = np.random.uniform(size=(2,4))
mat_b = np.random.uniform(size=(4,3))

mat_a

In [ ]:
mat_a@mat_b

It also throws an error if the multiplication can't happen.

Try `mat_b@mat_a`

In [ ]:
# mat_b@mat_a

You can also use `np.dot` to do the multiplication. However, be careful with this, as it also computes the actual dot product of vectors. It tries to be smart about how it does that, so it usually does a good job. 

In [ ]:
np.dot(mat_a, mat_b)

Vector dot product example:

In [ ]:
a = np.array([1,2,3])
b = np.array([4,5,6])
np.dot(a,b)

`numpy` also knows about basic matrix stuff like transposes, etc. It also has a ton of cool linear algebra stuff under `np.linalg` but we're going to skip over that for now.

In [ ]:
mat_a

In [ ]:
mat_a.T

### Vectorization

And here's why `numpy` is a magical beast. We can take many common things and convert them into a "vectorized" process - meaning that we do the same thing over the whole dataset using advanced broadcasting. So for instance, let's look at adding a vector to a matrix (we're going to add this vector to each row) 

In [ ]:
mat_a.T + np.array([1,2])

`numpy` noticed that the column lengths matched and automatically adjusted to adding the whole vectors.

What do we think the below code will do?

In [ ]:
mat_a - np.array([1,2,3,4])

We can use that to do things like calculating the distance between a single vector and a whole table of vectors. Let's see that in action.

**Let's find the vector distance between a bunch of vectors and one vector**

In [ ]:
new_vec = np.array([1,2])
all_vecs = np.random.uniform(size=(100,2))

If we were doing it for one vector:

In [ ]:
x = all_vecs[0]
np.sqrt((new_vec[0] - x[0])**2 + (new_vec[1] - x[1])**2)

Now let's do it for all 100 vectors:

In [ ]:
np.sqrt(np.sum((all_vecs - new_vec)**2, axis=1))[:10]

Many, many, many processes can be vectorized... and it can be a huge time saver on large datasets. Let's see the difference.

In [ ]:
%%timeit
output = []
for x in all_vecs:
    dist = np.sqrt((new_vec[0] - x[0])**2 + (new_vec[1] - x[1])**2)
    output.append(dist)

In [ ]:
%%timeit
np.sqrt(np.sum((all_vecs - new_vec)**2, axis=1))

What about at a larger scale than 100?

In [ ]:
all_vecs = np.random.uniform(size=(10000,2))

In [ ]:
%%timeit
output = []
for x in all_vecs:
    dist = np.sqrt((new_vec[0] - x[0])**2 + (new_vec[1] - x[1])**2)
    output.append(dist)

In [ ]:
%%timeit
np.sqrt(np.sum((all_vecs - new_vec)**2, axis=1))

In [ ]:
import time
list_timing = []
for num_vecs in np.linspace(10,10000,100):
    all_vecs = np.random.uniform(size=(int(num_vecs),2)).tolist()
    # To measure when we start our process
    start = time.time()
    
    # The actual process
    output = []
    for x in all_vecs:
        dist = np.sqrt((new_vec[0] - x[0])**2 + (new_vec[1] - x[1])**2)
        output.append(dist)
        
    # Figure out when the process was done and 
    # keep track of how long it took
    end = time.time()
    list_timing.append((num_vecs, end - start))

In [ ]:
array_timing = []
for num_vecs in np.linspace(10,10000,100):
    all_vecs = np.random.uniform(size=(int(num_vecs),2))
    # To measure when we start our process
    start = time.time()
    
    # The actual process
    output = np.sqrt(np.sum((all_vecs - new_vec)**2, axis=1))
        
    # Figure out when the process was done and 
    # keep track of how long it took
    end = time.time()
    array_timing.append((num_vecs, end - start))

In [ ]:
plt.figure(dpi=150)
list_X, list_times = zip(*list_timing)
array_X, array_times = zip(*array_timing)

plt.plot(list_X, list_times, 'r', label="Lists")
plt.plot(array_X, array_times,'b', label='Arrays')
plt.xlabel("Number of Vectors")
plt.ylabel("Computation Time")
plt.legend();

Hard to see what's happening there. Let's look on a log scale.

In [ ]:
plt.figure(dpi=150)
list_X, list_times = zip(*list_timing)
array_X, array_times = zip(*array_timing)

plt.plot(list_X, list_times, 'r', label="Lists")
plt.plot(array_X, array_times,'b', label='Arrays')
plt.yscale('log')
plt.xlabel("Number of Vectors")
plt.ylabel("Computation Time")
plt.legend();